# Basic concepts

## Spark Architecture

First, you must know that Spark is complex. Not only it includes a lot of code for doing many data processing tasks, but it also promises to be highly efficient and error free while executing code distributed across hundreds of machines. You don’t really need this complexity at the beginning, what you need is to abstract things away and forget the gritty details.

Something important though is the distinction between the _driver_ and the _workers_. The **driver** is the Python process where you issue the orders for workers. The **workers** are Spark processes that follow the driver’s orders. You might think of the workers as being in the cloud (the Spark cluster).

Spark assumes that big data will be distributed among the workers which together have enough memory and processing capacity to deal with it. The driver is not expected have enough resources to hold this amount of data, which is why it is necessary to explicitly say when to move data to the driver. However, care must be taken with the amount of data you brought to the driver.

In [1]:
from pyspark.sql import SparkSession

session = SparkSession.builder.getOrCreate()

## RDDs

The **RDD**s (Resilient Distributed Datasets) are one of the most important data structures in Spark, and the basis of dataframes. You can think of them as “distributed” arrays. In many regards they behave like lists, with a few details we’ll discuss bellow.

So, how to create an RDD? The most straightforward way is to _parallelize_ a Python array.

In [2]:
rdd = session.sparkContext.parallelize([1,2,3])

In [3]:
rdd.take(num=2)

[1, 2]

This will bring the first 2 values of the RDD to the driver. The `count` method will return the length of the RDD

In [4]:
rdd.count()

3

If you want to send all the RDD data to the driver as an array, you can use `collect`.

In [5]:
rdd.collect()

[1, 2, 3]

As mentioned before, collecting from the driver must be performed cautiously. There is a possibility that it could collapse the driver depending on the **RDD** size. In general, use `take` to partition the rows required and invoke `collect` only after assessing the **RDD** size.

## Dataframes

If you know _pandas_ or _R_ dataframes, you’ll have a very good idea of what Spark dataframes stand for. They represent tabular (matrix) data with named columns. Deep inside, they are implemented with an **RDD** of Row objects, which are somewhat similar to a tuple in Python. Similarities to _SQL_ makes it a powerful tool for those who are familiar. We’ll talk about dataframe manipulation later, but let’s start creating a dataframe so you can play with it.

In [6]:
df = session.createDataFrame(
  [[1,2,3], [4,5,6]], ['column1', 'column2', 'column3']
)

> The data matrix comes first and the column names later.

You can try the `take`, `count` and `collect` methods; `take` and `collect` will give you a list of Row objects. However, it may be easier to use `show`.

In [7]:
df.show(n=3)

+-------+-------+-------+
|column1|column2|column3|
+-------+-------+-------+
|      1|      2|      3|
|      4|      5|      6|
+-------+-------+-------+



This will print a table representation of the dataframe with the first n rows.

## Immutability

One key difference with Python lists is that RDDs, (and also dataframes), are _mmutable_. _Immutable_ data is often required in concurrent applications and functional languages.

In [8]:
a = list(range(10))
a.append(11)

> Here, the interpreter first creates a name `a` that points to a list object and in the second line it modifies that very same object.

In [37]:
st = 'my string'
st += ' is pretty'

The interpreter first creates the name `st` that points to a string object, then it creates a brand new string object (at a different location in memory) with "my string is pretty" and points the name `st` to that new object. This happens because strings are _immutable_.

Similarly, **RDD**s and dataframes can’t be modified in place.

In [38]:
rdd.map(lambda x: x*100)

PythonRDD[35] at RDD at PythonRDD.scala:53

> Here, `rdd` will not change.

In [39]:
my_rdd = rdd.map(lambda x: x*100)

> 'my_rdd' points to the transformed RDD object.

## Transformations and Actions

A more shocking difference with ordinary Python might confuse you at the beginning. Sometimes you’ll notice that a very heavy operation happens instantly. But later, you do something small (like printing the first value of the RDD) and it seems to take forever.

In Spark, there’s a distinction between transformations and actions. When you change a dataframe that’s a transformation, however, when you actually consume the data (e.g. df.show(1)) that’s an action. Transformations are lazy loaded, they don’t run when you call them. They are executed when you consume their results via an action. Then all the needed transformations will be planned, optimized and run together. So when you see operations that look instantaneous, even though they’re heavy, it’s because they are transformations and they’ll be run later.

# Dataframe Manipulation

## UDFs (User Defined Functions)

**User Defined Functions** let you use Python code to operate on dataframe cells. You create a regular Python function, wrap it in a UDF object and pass it to Spark, it will care of making your function available in all the workers and scheduling its execution to transform the data.

In [10]:
import pyspark.sql.functions as funcs
import pyspark.sql.types as types

In [11]:
def multiply_by_ten(number):
    return number*10.0

In [12]:
multiply_udf = funcs.udf(multiply_by_ten, types.DoubleType())
transformed_df = df.withColumn(
    'multiplied', multiply_udf('column1')
)

In [13]:
transformed_df.show()

+-------+-------+-------+----------+
|column1|column2|column3|multiplied|
+-------+-------+-------+----------+
|      1|      2|      3|      10.0|
|      4|      5|      6|      40.0|
+-------+-------+-------+----------+



First you create a Python function, it could be a method in an object, that’s also a function. Then you create an UDF object. The annoying part is that you need to define the output type, something we aren’t really used to in Python. To be really effective with UDFs you’ll need to learn those types, specially the composite MapType (like dictionaries) and ArrayType (like lists). The benefit is that then you can pass this UDF to the dataframe, tell it which column it will be operating on, and you’ll get fantastic things done without leaving the comfort of your old Python.

One of the main limitations with UDFs, though, is that although they can take several columns as input, they can’t change the row as a whole. If you want to work with the whole row, you’ll need the RDD map.

## RDD Mapping

It’s a lot like using a UDF, you also pass a regular Python function. But in this case, the function will be receiving a full Row object instead of column values. It will be expected to return a full Row as well. This will give you the ultimate power over your rows, with a couple of caveats. First: Row object are immutable, so you need to create a whole new Row and return it. Second: you need to convert the dataframe to an RDD and back again. Fortunately neither of these problems are hard to overcome.

Let me show you a function that will logarithmically transform all the columns in your dataframe. As a nice touch it also transforms each column name as ‘log(column_name)’. I find the easiest to start by getting the row into a dictionary with row.asDict(). The function has a bit of Python magic like dictionary comprehensions and key-word arguments packing (the double stars). Hope you understand all of it.

In [14]:
def take_log_in_all_columns(row: types.Row):
    
    old_row = row.asDict()
    new_row = {f'log({column_name})': math.log(value) for column_name, value in old_row.items()}
    
    return types.Row(**new_row)

In [16]:
log = df.rdd.map(take_log_in_all_columns)
log_df = session.createDataFrame(log)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 9.0 failed 1 times, most recent failure: Lost task 1.0 in stage 9.0 (TID 27) (f4b329f1c91a executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1562, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_7168/2

You’ll notice this is a chained method call. First you call rdd, it will give you the underlying RDD where the dataframe rows are stored. Then you apply map on this RDD, where you pass your function. To close you call toDF() that transforms an RDD of rows into a dataframe. See this Stack Overflow question for further discussion.

## SQL Operations

Since dataframes represent tables, naturally they are endowed with SQL-like operations. I’ll be referring just a few of them to get you excited, but you can expect to find almost isomorphic functionality.

Calling `select` will return a dataframe with only some of the original columns.

In [83]:
df1 = df.select('column1', 'column2')

This call to where will return a dataframe with only the rows where the value for column1 is 3.

In [84]:
df1.where('column1 = 3')

DataFrame[column1: bigint, column2: bigint]

This call to join will return a dataframe that is, well…, a join of df and df1 through column1 in the same way INNER JOIN would do in SQL.

In [85]:
df.join(df1, ['column1'], how='inner')

DataFrame[column1: bigint, column2: bigint, column3: bigint, column2: bigint]

In case you need to perform right or left joins, in our example df is like the left table and df1, the right one. Outer joins are also possible.

But on top of that, in Spark you can execute SQL much more directly. You can create a temporal view out of a dataframe

In [86]:
df.createOrReplaceTempView('table1')

And then perform a query on top of that view.

In [87]:
df2 = session.sql("SELECT column1 AS f1, column2 as f2 from table1")

These queries will return a new dataframe with the corresponding column names and values.

## Dataframe Column Operations

The column abstractions supports several direct operations like arithmetic, for instance, say you want to add column1 plus the multiplication of colum2 and column3.

In [92]:
df3 = df.withColumn(
    'derived_column', df['column1'] + df['column2'] * df['column3']
)

## Aggregations and Quick Statistics

For this example we will need to read from a CSV file. I downloaded the adult.data file from https://archive.ics.uci.edu/ml/datasets/adult.

I moved the file to the data folder in my repository and renamed it to adult.data.csv. I copied the column names from adult.names into a list:

In [94]:
ADULT_COLUMN_NAMES = [
     "age",
     "workclass",
     "fnlwgt",
     "education",
     "education_num",
     "marital_status",
     "occupation",
     "relationship",
     "race",
     "sex",
     "capital_gain",
     "capital_loss",
     "hours_per_week",
     "native_country",
     "income"
 ]

After reading from the file I set the column names using this list, since the file lacks headers.

In [95]:
csv_df = session.read.csv(
     'adult.data.csv', header=False, inferSchema=True
 )

for new_col, old_col in zip(ADULT_COLUMN_NAMES, csv_df.columns):
     csv_df = csv_df.withColumnRenamed(old_col, new_col)

After that you can try the following for some quick descriptive statistics.

In [97]:
csv_df.describe()

DataFrame[summary: string, age: string, workclass: string, fnlwgt: string, education: string, education_num: string, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: string, capital_loss: string, hours_per_week: string, native_country: string, income: string]

To get an aggregation use groupBy together with the agg method. For instance, the following will get you a dataframe with the work hours average and standard deviation by age group. We also sort the dataframe by age.

In [98]:
work_hours_df = csv_df.groupBy(
    'age'
).agg(
    funcs.avg('hours_per_week'),
    funcs.stddev_samp('hours_per_week')
).sort('age')

## Connecting to databases

It’s very likely that you keep your data in a relational database, handled by a RDBMS like MySQL or PostgreSQL. If that’s the case, don’t worry, Spark has ways of interacting with many kinds of data storage. I bet you can google your way through most IO needs you may have.

To connect to a RDBMS you need a JDBC driver, that would be a jar file Spark can use to talk to the database. I will show you a PostgreSQL example.

First you need to download the driver.

In my GitHub repo I put the JDBC driver in the bin folder, but you can have it in any path. Also, you should know the code in this example is at a different file in the repo. I had to do this because unlike the other file, this one will throw errors, as the database credentials are of course fake. You’d need to replace them with your database’s to make it run.

Now you’ll start Pyspark with an extra step:

In [99]:
session = SparkSession.builder.config(
    'spark.jars', 'bin/postgresql-42.2.16.jar'
).config(
    'spark.driver.extraClassPath', 'bin/postgresql-42.2.16.jar'
).getOrCreate()

This creates a session aware of the driver. You can then read from the database like this (replacing the fake configs with your real ones):

In [100]:
url = f'jdbc:postgresql://your_host_ip:5432/your_database'
properties = {'user': 'your_user', 'password': 'your_password'}

In [101]:
df = session.read.jdbc(
    url=url, table='your_table_name', properties=properties
)

Py4JJavaError: An error occurred while calling o549.jdbc.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:33)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:350)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:274)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:245)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader

Then you can create a transformed dataframe any way you want and write the data back to the database (maybe at a different table).

In [102]:
transformed_df.write.jdbc(
    url=url, table='new_table', mode='append', properties=properties
)

Py4JJavaError: An error occurred while calling o554.jdbc.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:218)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:222)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:46)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.

The writing modes according to the documentation are:

**append**: Append contents of this DataFrame to existing data.
**overwrite**: Overwrite existing data.
**ignore**: Silently ignore this operation if data already exists.
**error** or **errorifexists** (default case): Throw an exception if data already exists.